# Create custom track to visualize depth of coverage

In this notebook we are going to:
    - Create a BED file
    - Use this BED file to extract the depth of coverage of a genome using *samtools bedcov*
    - Use the output to create a wiggle custom track to visualize the depth of coverage in the UCSC browser
    - Extract the depth of coverage from a second genome to be used as control
    - Divide both coverages and create a wiggle custom track
    - Identify a Copy Number Variant
    - Find discordant reads and splitreads

First, we will import some libraries

In [1]:
import numpy as np
import pandas as pd

Next, we will define the chromosome, and the start and end of our region of interest. For this exercise, we will use chr1:25,000,000-26,000,000. We also need to define the size of the blocks to count coverage (as number of bases per block). In our case, we will use 1,000 (we will count the total number of bases of reads mapped to those 1,000 bases intervals, from chr1:25,000,000 to chr1:26,000,000). 

In [2]:
chr = 5
start = 180200000
end = 180600000
step = 1000

Now, we will create a range from the start to the end, using step. This will create an array (*start_pos*) with all starting positions.

In [3]:
start_pos = np.arange(start,end,step)

And we will iterate through every starting position in our array to print chromosome, starting position, and end position (starting position+1000). We will use the tab (*"\t"*) separator, so each one of the three fields is separated by tabs.

In [4]:
for i in start_pos:
    print(chr,i,i+1000,sep="\t")

5	180200000	180201000
5	180201000	180202000
5	180202000	180203000
5	180203000	180204000
5	180204000	180205000
5	180205000	180206000
5	180206000	180207000
5	180207000	180208000
5	180208000	180209000
5	180209000	180210000
5	180210000	180211000
5	180211000	180212000
5	180212000	180213000
5	180213000	180214000
5	180214000	180215000
5	180215000	180216000
5	180216000	180217000
5	180217000	180218000
5	180218000	180219000
5	180219000	180220000
5	180220000	180221000
5	180221000	180222000
5	180222000	180223000
5	180223000	180224000
5	180224000	180225000
5	180225000	180226000
5	180226000	180227000
5	180227000	180228000
5	180228000	180229000
5	180229000	180230000
5	180230000	180231000
5	180231000	180232000
5	180232000	180233000
5	180233000	180234000
5	180234000	180235000
5	180235000	180236000
5	180236000	180237000
5	180237000	180238000
5	180238000	180239000
5	180239000	180240000
5	180240000	180241000
5	180241000	180242000
5	180242000	180243000
5	180243000	180244000
5	180244000	180245000
5	18024500

The problem with our output above is that it contains 1,000 lines, and we would need to copy it and paste it in our Linux terminal. Maybe it might be more convenient to export the results to a file. Let's redo the command including a few lines of code to save the output to a file. What does the cell below do?
- First, it opens a file for writting, and it is assigned to an object call *f*.
- The next two lines are equivalent to the previous cell (you iterate over each position, and for each starting position, you create a tuple containing the chromosoe number, the start position, and the end position).
- The fourth line is the one that actually writes the output to a file:
    - We use the command f.write to write, and inside the parenthesis what we want to write.
    - We have a tuple (*res*) so we iterate over each element.
    - We convert them to strings (str(x)).
    - And we want fields to be separated by tabs, so we joined the elements using a tab (*'\t'*).

In [5]:
f = open('my_positions_na.bed', 'w')
for i in start_pos:
    res = (chr,i,i+1000)
    f.write('\t'.join(str(x) for x in res)+'\n')
f.close()

Now that you have your bed file (*my_positions.bed*), you can open you Linux terminal, and copy the bed file to your *home*. The path to write here is different depending on your system
```console
cp /mnt/c/Users/Your\ User/jupyter_notebooks/my_positions.bed .
```

And now you can use *samtools bedcov* to extract the bed coverage for a genome from the 1000genomes (HG00096) and save the results in HG00096.bedcov:
```console
    samtools bedcov my_positions.bed ftp://ftp-trace.ncbi.nih.gov/1000genomes/ftp/phase3/data/HG00096/high_coverage_alignment/HG00096.wgs.ILLUMINA.bwa.GBR.high_cov_pcr_free.20140203.bam  > HG00096.bedcov
```
Extracting the coverage for 1 Mb in steps of 1 Kb using a remote BAM will take a while ($\approx$40 min). Once done, you can copy the results back to your directory:
```console
cp HG00096.bedcov /mnt/c/Users/Your\ User/jupyter_notebooks/
```
And you should have the file in your jupyter notebooks directory. We are going to import the data:

In [6]:
cov = pd.read_csv("HG00096.bedcov_na",sep='\t',header=None)
cov_df = pd.DataFrame(cov)

You can take a quick look at the results:

In [7]:
cov_df

,0,1,2,3
0,5,180200000,180201000,52356
1,5,180201000,180202000,51276
2,5,180202000,180203000,58793
3,5,180203000,180204000,70377
4,5,180204000,180205000,47026
5,5,180205000,180206000,47567
6,5,180206000,180207000,63224
7,5,180207000,180208000,54735
8,5,180208000,180209000,46491
9,5,180209000,180210000,48680


You see that there are 1,000 rows and 4 columns, as we were expecting. The columns are not labeled, but they are chromosome, start and end positions, and the 4<sup>th</sup> column is the number of bases in that region. You can see that there is some variability between each 1,000 region. We can explore the file using the *describe* function for a pandas dataframe:

In [8]:
cov_df.describe()

,0,1,2,3
count,400.0,4.000000e+02,4.000000e+02,400.000000
mean,5.0,1.803995e+08,1.804005e+08,47279.962500
std,0.0,1.156143e+05,1.156143e+05,13105.904535
min,5.0,1.802000e+08,1.802010e+08,10504.000000
25%,5.0,1.802998e+08,1.803008e+08,39106.500000
50%,5.0,1.803995e+08,1.804005e+08,47958.500000
75%,5.0,1.804992e+08,1.805002e+08,55328.000000
max,5.0,1.805990e+08,1.806000e+08,109563.000000


If we just look at the 4<sup>th</sup> column, we can see that the mean is 45,868 bases, but there is a minimum of 13,620 and a maximum of 77,959. That means that, if we have near 46,000 bases of sequence per each 1,000 base interval, our depth of coverage is 46$\times$. That coverage is higher than usual for a whole genome, but is the one used for the 1000genomes project high coverage. What we are going to do now, is to create a *wiggle* track to visualize the coverage in the UCSC genome browser. In this wiggle track, we will use:
- The *fixedStep* format, in which we define in which chromosome and position do we start, and which is our step (in our case was 1,000 bases).
- Then we just put the values (the number of bases), and the track knows that each one of them is 1,000 bases apart from the previous one.
- We are going to define also the span to 1,000 bases. The wiggle will create a histogram, and with the span we are telling the track that we want the base to be 1,000 bases.
- We will export everything to a file called HG00096.wig that we will later upload to the UCSC genome browser.

In [9]:
f = open('HG00096_na.wig', 'w')
f.write('browser position chr5:180200000-180500000' + '\n')
f.write('track type=wiggle_0 name="fixedStep" description="fixedStep format" visibility=full autoScale=off viewLimits=0.0:109000.0 color=50,150,255 yLineMark=47000 yLineOnOff=on priority=10' + '\n')
f.write('fixedStep chrom=chr5 start=180200000 step=1000 span=1000' + '\n')
for index, row in cov_df.iterrows():
    f.write(cov.loc[index,3].astype(str) + '\n')
f.close()

Now we are ready to visualize the coverage of this genome in this region of chromosome 1. We'll go to the UCSC genome browser (http://genome.ucsc.edu) and:
- select **human assembly GRCh37/hg19** (because this is the version that the 1000genomes mapped the HG00096 reads to) and click **Go**.
- Right below the main window, you'll find a button saying "**add Custom tracks**", click on it.
- To upload the track we have just created, click on **Browse...** and select the file from your computer (*HG00096.wig*).
- Click **Submit**.
- You should get a track called *fixedStep*, type *wiggle_0*. To go an view it click on **go**.
You should now see chr1:25,000,000-26,000,000 with the track on blue on the top, such as in the image below:
![title](05_coverage_01v3.png)

The first thing you notice is that the coverage is very noisy. This is due to several reasons, including the GC content (regions with a high GC content are more difficult to PCR-amplify, so you usually have lower coverage), to the presence of repetitive regions,...
In order to smooth our track, we are going to click on the **grey bar on the left of the track**, and a menu with several options will appear. We are going to perform the following modifications:
- We are going to change the **smoothing window** to 16.
- As we know that the average number of bases is 45,868, we are going to **Draw y indicator lines:** at **y=45868**.
- Click **Submit**.
Now, we should have a more smoothed coverage track, as that in the figure below:
![title](05_coverage_02v3.png)
You can see that the coverage track is now smoothed, and most of this Megabase of sequence has the same coverage, with the exception of a small region around 25,600,000, in which we can see a drop in coverage to about half of the rest of this region. Therefore, it looks like there is a deletion affecting this region. If you take a look at the track just below the coverage, you can see that this deletion only affects one gene, named *RHD*. Click on that gene to see what it codes for.
Do you recognize this gene? Of course, it encodes *Homo sapiens* Rh blood group, D antigen, responsible for the Rh blood group. As you know, people are either Rh+ or Rh-, depending on whether they have this antigen or not. Rh- persons do not have a functional copy of the *RHD* gene, while Rh+ persons have it. As we are diploid, we have 2 copies of each chromosome (and usually, of all genes within that chromosome). In the case of the sample we are analyzing (HG00096), it appears that this person is diploid for most of the region we are exploring, with the exception of *locus RHD*, which has half the coverage than the other genes. This means that instead of having two copies of this gene, it only has one. Therefore, this person is Rh+, as at least has one copy of the gene, but is heterozygous for this deletion.

This *locus* allows us to try to identify the breakpoints, because one of the alleles should present a big deletion involving the *RHD* gene. First, we will zoom in into this region to more precisely see where the deletion starts and ends. You can see a zoomed image as that shown below.
![title](05_coverage_03v3.png)

The coverage is still noisy. We can see that the drop in coverage goes between 25,590,000 and 25,650,00, although due to the noise, those limits are not very precise. According to the 1000genomes, this CNV goes from 25,592,642 to 25,661,222, very close to the limits we obtained by visual inspection. We could try to improve the resolution and even try to identify the precise breakpoints by using the mate-pair information and the CIGAR. Unfortunately, this region is very repetitive, and together with the low coverage of 1000genomes sequences, this region is not suitable to this kind of analysis.

Luckily, there are other CNVs we can take advantage of to learn how to process this information. One such CNV is a deletion in chr1:72,766,343-72,811,815 (45 Kb). We will try to identify the breakpoints with higher accuracy by analyzing the BAM file, and more specifically, the mate-pair information.

To do that:
- First, we will view the BAM file for that region using *samtools view*, and we will use *awk* to select only those reads with an apparent insert size larger than 1,000. As the fragments insert sizes are about 500 bp, we will view the region extending 1,000 bases to each side:
```console
samtools view ftp://ftp-trace.ncbi.nih.gov/1000genomes/ftp/phase3/data/HG00096/high_coverage_alignment/HG00096.wgs.ILLUMINA.bwa.GBR.high_cov_pcr_free.20140203.bam 1:72765343-72812815|awk '$9>1000' > HG00096.chr1_72.sam
```
- You can explore the output file by using *less HG00096.chr1_72.sam* (**NOTE**: To quit *less*, just type 'q').
- The problem is that the output is too dense. So, in order to simplify the output, we will use *awk* to just print the most informative columns. In your terminal 
```console
awk '{print $3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9}' HG00096.chr1_72.sam
```
- Now, you should get a result as the one shown below:
```console
1       72765712        60      250M        =       203693994       130928076
1       72765819        60      250M        =       72811840        46237
1       72765987        60      250M        =       72811840        46097
1       72765993        59      184M66S     =       72811994        46251
1       72766001        60      250M        =       72811860        46109
1       72766008        60      250M        =       72811840        46054
1       72766091        60      234M16S     =       72811840        45886
1       72766092        60      233M17S     =       72811840        45886
1       72766120        60      205M45S     =       72811838        45968
1       72766124        60      201M49S     =       72811955        46081
1       72766137        60      188M62S     =       72811840        45894
1       72766150        60      175M75S     =       72811838        45938
1       72766150        60      157M93S     =       158166826       85400521
1       72766151        60      174M76S     =       72811918        46017
1       72766156        60      169M81S     =       72811840        45916
1       72766180        60      145M105S    =       72811888        45958
1       72766203        60      122M128H    =       72811855        45902
1       72766209        60      116M134H    =       72811931        45972
1       72766234        60      87M163H     =       72812283        46299
1       72766238        60      87M163H     =       72812214        46226
1       72766240        60      85M165H     =       72811890        45900
1       72766261        60      64M186H     =       72812041        46030
1       72766288        60      37M213H     =       72812068        46030
1       72768731        60      250M        =       193831512       121062829
1       72784230        0       5S63M182S   =       91505639        18721410
1       72786789        60      51S77M122S  =       168739245       95952457
1       72793697        0       59S25M166S  =       223453796       150660325
1       72803303        60      63S187M     =       142841358       70038060
1       72803545        0       90H40M120H  =       226526069       153722486
1       72805742        60      250M        =       106203355       33397365
1       72811860        56      194H56M     =       158166826       85354912
```

- On the right-most column we can see the apparent insert size. You can see that there are different sizes, but there is one size that appears to be very frequent. From the 2$^{nd}$ to the 23$^{rd}$ lines, all of them but one have an apparent insert size of about 46,000 bases.
- If we take a look at the second line, the read maps at chromosome 1, position 72,765,819, while its mate maps at 72,811,840 (that correspond more or less where the deletion should be).
- All the other lines with an apparent insert size of 46,000, map nearby, and the mate also nearby to the mate of the second read, suggesting that we have many independent fragments from the DNA library that happen to support this event.
- If we have fragments of 500 bp, and a fragment contains the breakpoint, the reads could not be more than 500 bases appart from the breakpoint. Thus, all independent reads supporting the deletion should map very close together. In our case, the first of those reads maps at position 72,765,819, while the last one at position 72,766,288, that means that they are 419 bases apart. Similarly, the mates map between 728,11,838 and 72,812,283, also 445 bases apart.
- By contrast, the other reads with apparent insert sizes larger than 1 Kb, are all very large, and there is only 1 read supporting each potential SV, suggesting that all of them constitute false positives.
- Finally, you can see that the mapping quality for all reads supporting the 46,000 deletion is very high (60), while for some of the reads we suspected they were false positives, the mapping quality is 0, meaning that our confidence that the read is properly mapped (and therefore supporting the presence of an SV) is very low, reafirming our suspicion of those reads being false positives.

In summary, we have 21 reads supporting the presence of a large deletion of about 46,000 bases (range 45,886-46,299). We can estimate that the breakpoint on the left is close to 72,766,288 (the right-most read), and the breakpoint on the left is close to 72,811,838.

Although the use of NGS has allowed us to narrow the positions of the breakpoints, we can try to be even more precise, defining the breakpoints at base-pair resolution. Considering the high coverage we have, and the fact that the reads lengths used are very long (250 bases), it is very likely that there are a couple of reads that cover the breakpoint. In this case, part of the read will belong to the left part of the breakpoint, while the rest of the read will belong to the sequence at the right-hand side (that is, 46,000 bases to the right). In this case, the alignment programs are not able to properly align the whole read. If part of the read aligns pretty well to a region, scanning the rest of the genome to try to find where the rest of the read aligns is very computing intensive, so it not usually done to keep the performance of the algorithm. In those cases, the read is align where there are more bases aligning, and then, it performs soft-clipping (the alignment will refer to the part that aligns, but not to the rest of the read). Those reads are known as splitreads. And those are precisely the ones we are interested in.

If we just take a look at the CIGAR, to see if any of those reads had soft-clipping, we can see that many of them are soft-clipped. For the sake of simplicity, we are going to show just one, the one with the largest fargment soft-clipped:
```console
1       72766180        60      145M105S    =       72811888        45958
```
Now, we will go to the *HG00096.chr1_72.sam* file to find this read:
```console
H7AGFADXX131213:2:1201:9933:68924       161     1       72766180        60      145M105S        =       72811888        45958   TGGATAAATATGTGAGGGAGACAATATAAAACAAAAGTTCACATGGAACATACATTAATAAAATACTGCAGAAGCAAAAAAAAGTGAATCATTCAAAAATGAAAAGGGTACAAGACAAAAATAAAAGTTAAAGAAAAACCAAATGACTGAGAGATTATCTCAGTGGATTTGACTTAATCTCTTGAACCACTTTCTAAAAACAGTTTTATTTGGCTAGTTGCAGAAAAGAAAATCAGAGATTCAAAGCTTT ;::>:>>@<>>>7=?>==?8><:<;<>>>>><????>>?>><>>><@?<>:;:><????>@@@>>=<<:9>?@>>@@@@@@@A@/:@??;A@A?AAAA@>@@AAA>=<??=@@?@>@B=?@??@@@??@?A@?@A@AA<@A?@>>A>@==>=8A?A@@@?=?>:9??=?@@@=A@@A?<>>A8>>?>=B?AA>@A?@@@?&?;@?AA@@8><>@<;?@+<9?>9>AA@BAA?>6@.+9+>A@AAA##### SA:Z:1,72811840,+,144S106M,60,3;   BD:Z:IIJIOIMFJGHKIIMLMIILKLHJIFGFJDDJHJDDKMJILHHIJJIIJHIFKHIHJJIFJDDIFKKMNNNKIKONJDDDDDDKMIMIILLIHILJDDDIJMIDDKMILKKHJKKLHJDDDIFJDDKMJJJDKKIDDDJMIJDIJMLKMMLKLKKHJGLJJLNMIJIKHCKMLKJJJILJJJJKMIJMIHKJCIJKJDDDJHNMJCCJGIDLKOOLNNKLOOOLJEELLJEEKNNPMNMNKLPNINIIII    RG:Z:H7AGF.2    BI:Z:KKKLPJNHLHILLKOMNLKLLLJMJHIHLGFLKMGGNOLKNJKLLKLKLKLHMKLKMLKHLGGKHMNPOPOMKNRPMGGGGGGNOKOKKNNLKKNMGGGKLOKGGNNLOMMKMNMMKMGGGKHLGGNOLMLGNMKGGGLNLMGKLPNNPPNNNNOLNJONMOPPLLMOLGNPNOMNMLONMNMNPLMOMKOMGLNPMHHHMLPPMHGOKMHOMRSQPQNOQRQOMIIPONJJNQQRPQQRPPSRLQKKKK    NM:i:0  BQ:Z:@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@    AS:i:145   XS:i:23
```
If you look carefully that information that the aligner provides, you can see that, right after the base quality information, there is one field with this information: *SA:Z:1,72811840,+,144S106M,60,3;*. The *SA* refers to secondary alignment, and the position where this read might also map is chr1:72,811,840. Right on the other breakpoint! The CIGAR is also provided (144S106M), meaning that if we map this read to this other locus, we need to soft-clip the first 144 bases of the read, and then the remaining 106 can be aligned. This more or less the opposite of what we have as primary alignment (CIGAR: 145M105S), in which the first 145 bases align, and the remaining 105 are soft-clipped.In addition we also have the mapping quality for the secondary alignment (60), so the confidence that, if we soft-clip the first bases we can get a good alignment, is high.

But this is what our aligner (BWA) says. Let's get a second opinion by using BLAT. If we just copy the sequence and perform a BLAT (remember to use GRCh37/hg19 to get the same coordinates than our genome), we should get the following results:
```console
  ACTIONS      QUERY   SCORE START   END QSIZE IDENTITY  CHROM  STRAND  START       END   SPAN
-----------------------------------------------------------------------------------------------
browser details YourSeq   243     1   250   250    98.8%  chr1   +    72766180  72811945  45766
browser details YourSeq    42   118   213   250    95.7%  chr3   +   186040363 186201121 160759
browser details YourSeq    23    99   124   250    96.0%  chr5   -    77769099  77769131     33
browser details YourSeq    23   117   143   250    88.0%  chr4   -    46220860  46220885     26
browser details YourSeq    23   126   148   250   100.0%  chr4   -    11167930  11167952     23
browser details YourSeq    23   186   208   250   100.0%  chr11  -    92916686  92916708     23
browser details YourSeq    22    21    44   250    87.0%  chr3   -      490524    490546     23
browser details YourSeq    22    85   107   250   100.0%  chr2   -   103296303 103296332     30
browser details YourSeq    22   219   240   250   100.0%  chr17  -    17370961  17370982     22
browser details YourSeq    21   118   138   250   100.0%  chr5   -   154514088 154514108     21
browser details YourSeq    21   118   138   250   100.0%  chr2   -   120875925 120875945     21
browser details YourSeq    21   118   138   250   100.0%  chr4   +    41765684  41765704     21
browser details YourSeq    21   118   138   250   100.0%  chr3   +   151746469 151746489     21
browser details YourSeq    20   119   138   250   100.0%  chr5   -    25550364  25550383     20
browser details YourSeq    20   119   138   250   100.0%  chr5   -    17836288  17836307     20
browser details YourSeq    20    28    49   250    95.5%  chr2   -   211122061 211122082     22
browser details YourSeq    20    28    49   250    95.5%  chr3   +   137116000 137116021     22
browser details YourSeq    20   118   137   250   100.0%  chr3   +    69691667  69691686     20
```

You can see that the first hit is the only one that is able to align the total length of the read (250 bases), and it aligns with the 98.8% identity (the remaining results only align part of the read, as little as 20 bases, so they don't look as credible results). However, if you take a look at the span, you can see that it does not span 250 bases (as it would be expected for a 250 bases read), but it spans 45,766 bases, which is exactly what you would expect if there was a deletion of $$45766-250 = 45516~bases$$ That means that this read lies right at the breakpoint, and the breapoint has been sequenced (in fact, it is this read). If we take a look at the BLAT alignment, we can see that the alignment is perfect, and you can define, at base-pair resolution, where the breakpoints are located, precisely at chr1:72,766,323 and at chr1:72,811,839. You could now design a pair of primers to try to validate this finding by an orthogonal technique such as Sanger sequencing.
```console
TGGATAAATA TGTGAGGGAG ACAATATAAA ACAAAAGTTC ACATGGAACA  72766229
TACATTAATA AAATACTGCA GAAGCAAAAA AAAGTGAATC ATTCAAAAAT  72766279
GAAAAGGGTA CAAGACAAAA ATAAAAGTTA AAGAAAAACC AAATgcaata  72766329
gtagaattaa aattcatatt ctaaatatta aaacataaaa acatacaagt  72766379
agtattgaca tttcagcaaa ttggataagt gatatggagg aaaaacatga  72766429
agaaatatcc agatacctga gggaaaaaaa tgtgatgtat aataagtatt  72766479
...
ctatctgagg tcctacagta aaaagtggag tatggtaggc agaatactaa  72811729
gatggcctgt caaggtttct cgaccatgtc attcattaaa acattaatct  72811779
aggtgctgtt gtgaagagat tttgttgatt taaatgaagt cccagcttgg  72811829
tagattttaa GACTGAGAGA TTATCTCAGT GGATTTGACT TAATCTCTTG  72811879
AACCACTTTC TAAAAAgAGT TTTAgTTGGC TAGTTGCAGA AAAGAAAATC  72811929
AGAGtTTCAA AGCTTTaagg ggattcaatg tgccactgat gacttgtaga  72811979
```

In summary, this is the general procedure to detect CNVs and other structural variants from NGS data. However, there are specialized programs to detect CNVs by taking into consideration not only the number of reads, but the GC content and mappability of the region, improving the ability to detect CNVs. Similarly, the detection of SVs based on the combination of abnormally paired reads (discordant reads) as well as splitreads, allows the detection of different types of SVs (including duplications, inversions, translocations,...)

**You can now try it yourself. You can use the same case, but try on chromosome 12, between 10,000,000 and 11,000,000.**